In [1]:
# packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
def calc_gender_dist_emp(JEDI_output,this_state,f_type):
    
    census_sex = pd.read_excel(r"../../data/demographics/census_sex_compile_5Y_2020.xlsx")
    JEDI_name = pd.read_excel(r"../../data/demographics/census_to_JEDI.xlsx",sheet_name = "JEDI")
    
    all_years = JEDI_output.columns
    JEDI_output["Type"] = "Total"

    census_sex_state = pd.merge(census_sex.query("State == @this_state"), JEDI_name, left_on = "JEDI_Index", right_on = "JEDI_Index")
    census_sex_state = census_sex_state.set_index("JEDI_Industries")

    s = "Female"
    JEDI_output_this = JEDI_output.copy()
    if f_type == "mid":
        JEDI_output_this["p"] = census_sex_state[s+"_Perc"]
    elif f_type == "ub":
        JEDI_output_this["p"] = census_sex_state[s+"_Perc"] + census_sex_state[s+"_Perc_MoE"]
    elif f_type == "lb":
        JEDI_output_this["p"] = census_sex_state[s+"_Perc"] - census_sex_state[s+"_Perc_MoE"]
    for y in all_years:
        JEDI_output_this[y] = JEDI_output_this[y]*JEDI_output_this['p']
    JEDI_output_this["Type"]= s
    JEDI_output_this = JEDI_output_this.drop(columns = "p")
    JEDI_output_all = JEDI_output.append(JEDI_output_this)

    f_jobs = JEDI_output_all.query("Type == 'Female'")
    f_jobs_sum = f_jobs.sum(axis=0)
    f_jobs_sum["Type"] = "Female"
    f_jobs_sum = f_jobs_sum.to_frame().transpose()

    t_jobs = JEDI_output_all.query("Type == 'Total'")
    t_jobs_sum = t_jobs.sum(axis=0)
    t_jobs_sum["Type"] = "Total"
    t_jobs_sum = t_jobs_sum.to_frame().transpose()

    all_jobs_sum = f_jobs_sum.append(t_jobs_sum).set_index("Type")
    all_jobs_sum = all_jobs_sum.transpose()
    all_jobs_sum["f_perc"] = all_jobs_sum["Female"]/all_jobs_sum["Total"]*100
    
    return all_jobs_sum